## GPT-3.5を使用したMoodleクイズ問題の生成

以下のセルのコードを上から順番に実行します。出力になにかおかしなところがあれば、そのセルを再度実行（=regenerate）し、その後のセルも再度実行してください。

Moodleの問題バンクにインポートできるMoodle XMLファイルが作成されます。

In [1]:
# openai python ライブラリーをインストールしていない場合はこれを実行してください。
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.5 MB/s eta 0:00:00


In [2]:
data1 = input("何についての問題を生成したい？（最後にエンターキーを）")

何についての問題を生成したい？（最後にエンターキーを）ブラックショールズ方程式


In [ ]:
import openai
openai.api_key = input("openai API key:")
model ="gpt-3.5-turbo-0613"
# model ="gpt-3.5-turbo"

In [4]:
prstr1 = '''
についての選択問題を2個作成してください。以下の条件を満たしてください。
・選択肢は5個
・問題文は丁寧な日本語（ですます調）で書かれている
・選択肢は丁寧ではない日本語（体言止めなど）や単語だけで書かれている
・どの選択肢が正解であるかが示されている
'''

prstr1 += data1
response = openai.ChatCompletion.create(
    model = model,
    messages=[
        {"role": "user", "content": prstr1},
    ],
)
res1 = response.choices[0]["message"]["content"].strip()
print(res1)

1. ブラックショールズ方程式に関する以下の選択肢のうち、正しいものはどれですか？
a) 株価の変動を予測するために使われる
b) オプション価格を計算するのに使われる
c) デリバティブ取引のリスクを測るために使われる
d) 株式市場のトレンドを予測するために使われる
e) 為替相場の変動を予測するために使われる

正解: b) オプション価格を計算するのに使われる

2. ブラックショールズ方程式は、以下の選択肢のうちどの分野で広く利用されていますか？
a) 経済学
b) 物理学
c) 数学
d) 科学技術
e) 化学

正解: a) 経済学


In [5]:
prstr2 = '''
今のすべての問題を一連のデータとして（問題ごとに分けずに） 以下のような Moodle XML 形式で出力してください。選択肢の句点は削除してください。

  <question type="multichoice">
    <name>
      <text>問いの要約</text>
    </name>
    <questiontext format="html">
      <text><![CDATA[問いの文章をここに書く。]]></text>
    </questiontext>
    <shuffleanswers>true</shuffleanswers>
    <answer fraction="0">
      <text>不正解1</text>
    </answer>
    <answer fraction="100">
      <text>正解</text>
    </answer>
    <answer fraction="0">
      <text>不正解2</text>
    </answer>
    <answer fraction="0">
      <text>不正解3</text>
    </answer>
    <generalfeedback>
      <text><![CDATA[全般に対するフィードバック]]></text>
    </generalfeedback>
  </question>
'''

response = openai.ChatCompletion.create(
    model = model,
    messages=[
        {"role": "user", "content": prstr1},
        {"role": "assistant", "content": res1},
        {"role": "user", "content": prstr2},
    ],
)

res2 = response.choices[0]["message"]["content"].strip()
print(res2)

<quiz>
  <question type="multichoice">
    <name>
      <text>ブラックショールズ方程式に関する問題1</text>
    </name>
    <questiontext format="html">
      <text><![CDATA[ブラックショールズ方程式に関する以下の選択肢のうち、正しいものはどれですか？]]></text>
    </questiontext>
    <shuffleanswers>true</shuffleanswers>
    <answer fraction="0">
      <text>株価の変動を予測するために使われる</text>
    </answer>
    <answer fraction="100">
      <text>オプション価格を計算するのに使われる</text>
    </answer>
    <answer fraction="0">
      <text>デリバティブ取引のリスクを測るために使われる</text>
    </answer>
    <answer fraction="0">
      <text>株式市場のトレンドを予測するために使われる</text>
    </answer>
    <answer fraction="0">
      <text>為替相場の変動を予測するために使われる</text>
    </answer>
    <generalfeedback>
      <text><![CDATA[ブラックショールズ方程式はオプション価格を計算するために使われます。]]></text>
    </generalfeedback>
  </question>
  
  <question type="multichoice">
    <name>
      <text>ブラックショールズ方程式に関する問題2</text>
    </name>
    <questiontext format="html">
      <text><![CDATA[ブラックショールズ方程式は以下の選択肢のうちどの分野で広く利用されていますか？]]></text>
    </que

In [6]:
prstr3 = '''
以下のように直してください。
・<quiz> から開始する
・選択肢番号があれば削除
・正解の選択肢の配点は 100 になっている
・問題名（<name><text> と </text></name> の間）は、問いの文章を10文字程度で要約したもの
・各問題に対して1つだけ、</questiontext> の後に、200文字程度の「全般的なフィードバック」を <generalfeedback><text> と </text></generalfeedback> で囲って書く
・「全般的なフィードバック」には、参考となるWikipediaページなどのリンクが target=_BLANK の指定付きで含まれている
・各問題に対して <shuffleanswers>true</shuffleanswers> を指定する
・[CDATA[ に対応する ]] がちゃんと書かれている
'''

response = openai.ChatCompletion.create(
    model = model,
    messages=[
        {"role": "user", "content": prstr2},
        {"role": "assistant", "content": res2},
        {"role": "user", "content": prstr3},
    ],
)
res3 = response.choices[0]["message"]["content"].strip()
print(res3)
# replace("^'''xml","",res3)
# replace("'''$","",res3)

with open('moodle-quiz-questions01.xml', 'w') as f:
    print(res3, file=f)

<quiz>
  <question type="multichoice">
    <name>
      <text>ブラックショールズ方程式に関する問題1</text>
    </name>
    <questiontext format="html">
      <text><![CDATA[ブラックショールズ方程式に関する以下の選択肢のうち、正しいものはどれですか？]]></text>
    </questiontext>
    <shuffleanswers>true</shuffleanswers>
    <answer fraction="0">
      <text>株価の変動を予測するために使われる</text>
    </answer>
    <answer fraction="100">
      <text>オプション価格を計算するのに使われる</text>
    </answer>
    <answer fraction="0">
      <text>デリバティブ取引のリスクを測るために使われる</text>
    </answer>
    <answer fraction="0">
      <text>株式市場のトレンドを予測するために使われる</text>
    </answer>
    <answer fraction="0">
      <text>為替相場の変動を予測するために使われる</text>
    </answer>
    <generalfeedback>
      <text><![CDATA[ブラックショールズ方程式はオプション価格を計算するために使われます。参考: <a href="https://ja.wikipedia.org/wiki/%E3%83%96%E3%83%A9%E3%83%83%E3%82%AF%E3%82%B7%E3%83%A7%E3%83%BC%E3%83%AB%E3%82%BA%E6%96%B9%E7%A8%8B%E5%BC%8F" target="_BLANK">Wikipedia</a>]]></text>
    </generalfeedback>
  </question>
  
  <question type="multich